### Install the necessary packages:

In [19]:
import G1_common_tools as tools
tools.install_required_packages()

sqlalchemy_utils is already installed.
Faker not found. Installing...
Faker has been installed.
fastapi[all] not found. Installing...
fastapi[all] has been installed.


### Set up the base of the API:

In [20]:
from fastapi import FastAPI
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey,func
from sqlalchemy.orm import sessionmaker, relationship, declarative_base, joinedload
from datetime import timedelta
import table_structures as t

# Database setup
DATABASE_URL = "sqlite:///it_ticketing_system.db"
Base = declarative_base()
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# FastAPI setup
app = FastAPI(
    title="IT Ticketing System API",
    description="API for Group 1's IT Ticketing System",
    version="0.1",
)

### API Definitions:

In [21]:
@app.get("/Technicians/")
def read_technician_names():
    '''
    Returns the first and last names of all the technicians
    '''
    db = SessionLocal()
    query = db.query(t.User).join(t.Technician).all()

    technicians = []
    for row in query:
        technician = row.first_name + ' ' + row.last_name
        technicians.append(technician)
    db.close()
    return technicians

@app.get("/Technicians/AvgTicketTimes/")
def read_technician_avg_ticket_times():
    '''
    TODO: Insert tooltip documentation here
    '''
    db = SessionLocal()
    query = db.query(t.Technician).join(t.TicketLine).join(t.User)
    query.all()
    technicians = []
    for row in query:
        ticket_durations = []
        for x in row.ticket_lines:
            interval = x.completion_date_time - x.assignment_date_time
            ticket_durations.append(interval)
        total_seconds = sum(interval.total_seconds() for interval in ticket_durations)
        average_seconds = total_seconds / len(ticket_durations) if ticket_durations else 0
        average_interval = timedelta(seconds=average_seconds)
        technician = row.user.first_name + ' ' + row.user.last_name + ' - Average Ticket Time: ' + str(average_interval)
        technicians.append(technician)
    db.close()
    return technicians


@app.get("/Users/TicketCounts/")
def read_user_ticket_counts(user_id=None):
    '''
    TODO: Insert tooltip documentation here
    '''
    db = SessionLocal()
    query = db.query(t.User).join(t.Ticket)
    user_data = []
    if not user_id:
        users = query.all()
    else:
        users = query.filter(t.User.user_id == user_id).all()

    for row in users:
        ticket_count = len(row.tickets)
        user = row.first_name + ' ' + row.last_name + ' Ticket Count: ' + str(ticket_count)
        user_data.append(user)

    db.close()
    return user_data


@app.get("/Departments/AvgResolutionTimes")
def read_department_avg_resolution_time():
    '''
    TODO: Retrieve and print the average resolution times for each department.
    '''
    db = SessionLocal()
    query = db.query(t.Department.name, func.avg(t.Ticket.close_date_time - t.Ticket.open_date_time).label('avg_resolution_time_minutes'))\
    .join(t.Ticket, t.Department.department_id == t.Ticket.department_id)\
    .group_by(t.Department.name)
    
    results = query.all()

    departments = []
    for department_name, avg_resolution_time_minutes in results:
        departments = department_name + ' ' + avg_resolution_time_minutes
        departments.append(departments)

    db.close()
    return departments

@app.get("/Technicians/TicketsInfo")
def read_technician_ticketinfo():
    '''
    TODO: Retrieve and print ticket information for each technician based on technician ID.
    '''
    db = SessionLocal()

    desired_technician_id = 1

    query = db.query(t.Ticket.ticket_id, t.Ticket.subject, t.Ticket.open_date_time, t.Ticket.close_date_time)\
    .join(t.TicketLine, t.Ticket.ticket_id == t.TicketLine.ticket_id)\
    .filter(t.TicketLine.technician_id == desired_technician_id)
    
    results = query.all()

    technicianticket = []
    for ticket_id, subject, open_date_time, close_date_time in results:
        technicianticket = ticket_id + ' ' + subject + ' ' + open_date_time + ' ' + close_date_time
        technicianticket.append(technicianticket)

    db.close()
    return technicianticket

@app.get("/Organizations/TicketCounts")
def read_organizations_tickets_count():
    '''
    TODO: Retrieve and print ticket counts for each organization.
    '''
    db = SessionLocal()

    query = db.query(t.Organization.name, func.count(t.Ticket.ticket_id).label('ticket_count'))\
    .outerjoin(t.Ticket, t.Organization.organization_id == t.Ticket.organization_id)\
    .group_by(t.Organization.name)
    
    results = query.all()

    organizationticket = []

    for organization_name, ticket_count in results:
        organizationticket = organization_name + ' ' + ticket_count
        organizationticket.append(organizationticket)

    db.close()
    return organizationticket
    

### API Calls

In [23]:
# read_technician_names()
# read_technician_avg_ticket_times()
# read_user_ticket_counts()
# read_user_ticket_counts(1)
# read_department_avg_resolution_time()
# read_technician_ticketinfo()
# read_organizations_tickets_count()
